In [370]:
import json

# Read data from the JSON file as text, specifying the encoding
with open('TrainingDataSet.json', 'r', encoding='utf-8') as file:
    json_text = file.read()

# Parse the JSON text
data = json.loads(json_text)

# Extract the 'annotations' from the JSON data
annotations = data['annotation']

# Define TRAIN_DATA as an empty list
TRAIN_DATA = []

# Iterate over each annotation and append it to TRAIN_DATA
for annotation in annotations:
    TRAIN_DATA.append(annotation)

# Print TRAIN_DATA to verify
print(TRAIN_DATA)


[['VAT: GB123 4567 89', {'entities': [[5, 18, 'VAT']]}], ['VAT Number: GB123 4567 89', {'entities': [[12, 25, 'VAT']]}], ['VAT: GB234 5678 90', {'entities': [[5, 18, 'VAT']]}], ['VAT Number: GB234 5678 90', {'entities': [[12, 25, 'VAT']]}], ['VAT: GB345 6789 01', {'entities': [[5, 18, 'VAT']]}], ['VAT Number: GB345 6789 01', {'entities': [[12, 25, 'VAT']]}], ['VAT: GB456 7890 12', {'entities': [[5, 18, 'VAT']]}], ['VAT Number: GB456 7890 12', {'entities': [[12, 25, 'VAT']]}], ['VAT: GB567 8901 23', {'entities': [[5, 18, 'VAT']]}], ['VAT Number: GB567 8901 23', {'entities': [[12, 25, 'VAT']]}], ['VAT: GB678 9012 34', {'entities': [[5, 18, 'VAT']]}], ['VAT Number: GB678 9012 34', {'entities': [[12, 25, 'VAT']]}], ['VAT: GB789 0123 45', {'entities': [[5, 18, 'VAT']]}], ['VAT Number: GB789 0123 45', {'entities': [[12, 25, 'VAT']]}], ['VAT: GB890 1234 56', {'entities': [[5, 18, 'VAT']]}], ['VAT Number: GB890 1234 56', {'entities': [[12, 25, 'VAT']]}], ['VAT: GB901 2345 67', {'entities': [[5

In [371]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en") # load a new spacy model
#nlp = spacy.load("en_core_web_sm") # load other spacy model

db = DocBin() # create a DocBin object
skipped_entities = []
for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            skipped_entities.append((text, start, end, label))
            print("Skipping entity:", text, start, end, label)
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

# Print skipped entities
print("Skipped entities:")
for entity in skipped_entities:
    print(entity)


current_folder = os.getcwd()
print("Current folder path:", current_folder)
os.chdir(current_folder)
db.to_disk("./train.spacy") # save the docbin object

100%|██████████| 40/40 [00:00<00:00, 4806.67it/s]

Skipped entities:
Current folder path: c:\Users\HP\Desktop\Project


In [ ]:
#Train your Model after this by running this command in command prompt in the project directory
# Command: python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

In [360]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
            text += "\n\n"  # Add two new lines after each page
    return text

# Path to your PDF file
pdf_file_path = "Test Files\PaySlip.pdf"

# Extract text from PDF
invoice_text = extract_text_from_pdf(pdf_file_path)

# Print the extracted text
print(invoice_text)


O'Hare & McGovern Ltd 
34 Whitehead, Belfast, BT306HG 
Phone: 08882589873 
Website: www.OHare&McGovern.co.uk 
 
Pay Slip 
 
Employee Information: 
Name: Siya Philip 
Address: 71 Meadowlands, Downpatrick, BT306HG 
National Insurance Number: WE234667Q 
 
Payment Information: 
Payment Date: 30 April 2024 
Payment Period: 1 April 2024 to 30 April 2024 
Gross Pay: £2500.00 
Tax Deductions: £500.00 
National Insurance Deductions: £200.00 
Net Pay: £1800.00 
 
Tax Year-to-Date: 
Gross Earnings: £12000.00 
Tax Paid: £2500.00 
National Insurance Paid: £1000.00 





In [361]:
nlp1 = spacy.load(r".\output\model-best") #load the best model
#invoice_text = "Please find the VAT number GB123 4567 89 on the invoice."
#doc = nlp1("My name is Siya Philip. My address is 11-16 Donegall Square East, Belfast, BT1 5UB. I have a friend named Hannah.My National insurance number is NZ368561C. My bank account number is 01581823. My sort code is 09-09-09. I work at BT Group.") # input sample text
doc = nlp1(invoice_text)

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter

In [297]:
#entity_dict = {}
for ent in doc.ents:
    #entity_dict[ent.label_] = ent.text
    print("Entity: {}, Label: {}".format(ent.text, ent.label_))
#print(entity_dict)

Entity: INV-35/29, Label: Invoice_Number


In [298]:
import re

classifications = {
    "Full_Name": r"([A-Z][a-z]+)\s([A-Z][a-z]+)(?:,\s([A-Z][a-z]+))?",
    "VAT": r"GB\d{3}\s\d{4}\s\d{2}",
    "ADDRESS": r"\d+\s[A-Za-z\s]+,\s[A-Za-z\s]+,\s[A-Z]{2}\d\s\d[A-Z]{2}",
    "Salary": r"£\d{1,3}(?:,\d{3})*\.\d{2}",
    "TaxCode": r"\d{4}[A-Za-z]",
    "BAN": r"\d{8}",
    "NI": r"[A-Z]{2}\d{6}[A-Z]",
    "Sort_Code": r"\d{2}-\d{2}-\d{2}",
    "ORG": r"([A-Z][a-z]+\s?)+",
    "Date": r"\d{1,2}\s?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)?\s?\d{2,4}",
    "Invoice_Number": r"INV-\d{2}/20\d{2}",
    "PhoneNumber": r"(?:\+?44-?)?\d{11}"
}

def match_text_with_label(text, label):
    if label in classifications:
        regex = classifications[label]
        match = re.fullmatch(regex, text)
        if match:
            return label, match.group()
        else:
            return None, None
    else:
        return None, None

# Test the function
for ent in doc.ents:
    #entity_dict[ent.label_] = ent.text
    print("Entity: {}, Label: {}".format(ent.text, ent.label_))
    #text = "John Doe"
    #label = "Full_Name"
    matched_label, matched_text = match_text_with_label(ent.text, ent.label_)
    if matched_label:
        print(f"Matched label: {matched_label}, matched text: {matched_text}")
    else:
        print("No match found.")



Entity: INV-35/29, Label: Invoice_Number
No match found.


In [283]:
from faker import Faker

# Initialize Faker object
faker = Faker()

# Sample input text
input_text = invoice_text
#"My name is Kiya Philip. My address is 11-16 Donegall Square East, Belfast, BT1 5UB. I have a friend named Hannah."

# Function to generate a fake National Insurance Number (NIN)
def generate_fake_nin():
    # Customize this function to generate a fake NIN based on the specific format and rules
    return faker.numerify(text="##########")

# Function to mask the data
def data_masked(doc):
    # Initialize empty string to store modified text
    modified_text = input_text
    # Initialize an empty dictionary to store the mapping between fake and original values
    fake_to_original_mapping = {}
    # Iterate over the identified entities
    for ent in doc.ents:
        label = ent.label_
        text = ent.text
        # Generate fake data based on the entity label
        if label == "FULL_NAME":
            fake_name = faker.name()
            # Replace the entity text with the fake name in the modified text
            modified_text = modified_text.replace(text, fake_name)
            # Store the mapping between the fake and original names
            fake_to_original_mapping[fake_name] = text
        elif label == "ADDRESS":
            fake_address = faker.address()
            # Replace the entity text with the fake address in the modified text
            modified_text = modified_text.replace(text, fake_address)
            # Store the mapping between the fake and original addresses
            fake_to_original_mapping[fake_address] = text
        elif label == "NATIONAL_INSURANCE_NUMBER":
            fake_nin = generate_fake_nin()
            # Replace the entity text with the fake NIN in the modified text
            modified_text = modified_text.replace(text, fake_nin)
            # Store the mapping between the fake and original NINs
            fake_to_original_mapping[fake_nin] = text

    # Print the masked data
    print("Masked Data:")
    print(modified_text)
    # Return the modified text and fake-to-original mapping
    return modified_text, fake_to_original_mapping

# Function to unmask the data
def data_unmasked(modified_text, fake_to_original_mapping):
    # Initialize empty string to store unmasked text
    unmasked_text = modified_text
    # Iterate over the fake-to-original mapping and replace the fake values with the original values
    for fake_value, original_value in fake_to_original_mapping.items():
        unmasked_text = unmasked_text.replace(fake_value, original_value)

    # Print the unmasked data
    print("\nUnmasked Data:")
    print(unmasked_text)

# Call data_masked() to show masked data and get modified_text and fake_to_original_mapping
modified_text, fake_to_original_mapping = data_masked(doc)

# Call data_unmasked() to show unmasked data
data_unmasked(modified_text, fake_to_original_mapping)


Masked Data:
Tax Invoice  
 
Invoice Number : INV -22/2024  
Tax Code : 1456G  
Date: 17 April 2024  
 
Bill to: 
Charles Wong488 Patterson Hill Apt. 293
South Amy, MI 71361  
 
Service Description : Consulting Services  
 
Amount Due : £500.00  
Seller Account Number: 01581823  
Sort Code: 40-50-60 
 
Payment Due Date : 30 April 2024  
 
Organis ation : Farrans Construction  
National Insurance Number : AB123456C  
VAT Number : GB345 6789 01  

Unmasked Data:
Tax Invoice  
 
Invoice Number : INV -22/2024  
Tax Code : 1456G  
Date: 17 April 2024  
 
Bill to: 
John Doe  
71 Meadowlands, Downpatrick, BT306HG  
 
Service Description : Consulting Services  
 
Amount Due : £500.00  
Seller Account Number: 01581823  
Sort Code: 40-50-60 
 
Payment Due Date : 30 April 2024  
 
Organis ation : Farrans Construction  
National Insurance Number : AB123456C  
VAT Number : GB345 6789 01  


In [118]:
# Define access levels for each combination of user designation and document type
access_levels = {
    "Admin": {"Payslip": "unmasked", "Tax Invoice": "unmasked"},
    "Manager": {"Payslip": "masked", "Tax Invoice": "unmasked"},
    "Employee": {"Payslip": "unmasked", "Tax Invoice": "masked"}
}

# User input: user designation and document type
designation = input("Enter your designation (Admin/Manager/Employee): ")
document_type = input("Enter the document type (Payslip/Tax Invoice): ")

# Check user access level based on designation and document type
access_level = access_levels.get(designation, {}).get(document_type, "masked")

# Call the appropriate function based on user's access level and document type
if access_level == "masked":
    if designation == "Employee" and document_type == "Payslip":
        # Call data_masked() with doc as input
        data_masked(doc)
    elif designation != "Employee":
        # Call data_masked() with doc as input
        data_masked(doc)
    else:
        print("You do not have access to view this document.")
else:
    # Call data_unmasked() with modified_text and fake_to_original_mapping as inputs
    data_unmasked(modified_text, fake_to_original_mapping)


You do not have access to view this document.
